# Running SAR on MovieLens

SAR is a fast scalable adaptive algorithm for personalized recommendations based on user transaction history and item descriptions. It produces easily explainable / interpretable recommendations and handles "cold item" and "semi-cold user" scenarios.

In [1]:
# set the environment path to find Recommenders
import sys
sys.path.append("..")

from utilities.recommender.sar import sar_singlenode
from utilities.dataset.url_utils import maybe_download
#TODO from utilities.dataset.splitters_python import
#TODO from utilities.evaluation.python_evaluation import PythonRankingEvaluation, PythonRatingEvaluation

import datetime
import itertools
import pandas as pd

First, let's download the MovieLens dataset:

In [34]:
filepath = maybe_download('http://files.grouplens.org/datasets/movielens/ml-100k/u.data', 'ml-100k.data')
data = pd.read_csv('ml-100k.data', sep='\t', names=['userid', 'itemid', 'rating', 'timestamp'])

In [35]:
display(data.head())

,userid,itemid,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


#### Next, we split the data using the provided Random Splitter utility function: